## 필요한 라이브러리 불러오기

In [1]:
import pandas as pd

from utils import load_dataset
from config import DATA_DIR

In [2]:
games = load_dataset('games')
ranking = load_dataset('ranking')
games.head(10)

,GAME_DATE_EST,GAME_ID,GAME_STATUS_TEXT,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,...,AST_home,REB_home,TEAM_ID_away,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
0,2022-12-22,22200477,Final,1610612740,1610612759,2022,1610612740,126.0,0.484,0.926,...,25.0,46.0,1610612759,117.0,0.478,0.815,0.321,23.0,44.0,1
1,2022-12-22,22200478,Final,1610612762,1610612764,2022,1610612762,120.0,0.488,0.952,...,16.0,40.0,1610612764,112.0,0.561,0.765,0.333,20.0,37.0,1
2,2022-12-21,22200466,Final,1610612739,1610612749,2022,1610612739,114.0,0.482,0.786,...,22.0,37.0,1610612749,106.0,0.470,0.682,0.433,20.0,46.0,1
3,2022-12-21,22200467,Final,1610612755,1610612765,2022,1610612755,113.0,0.441,0.909,...,27.0,49.0,1610612765,93.0,0.392,0.735,0.261,15.0,46.0,1
4,2022-12-21,22200468,Final,1610612737,1610612741,2022,1610612737,108.0,0.429,1.000,...,22.0,47.0,1610612741,110.0,0.500,0.773,0.292,20.0,47.0,0
5,2022-12-21,22200469,Final,1610612738,1610612754,2022,1610612738,112.0,0.386,0.840,...,26.0,62.0,1610612754,117.0,0.469,0.778,0.462,27.0,47.0,0
6,2022-12-21,22200470,Final,1610612751,1610612744,2022,1610612751,143.0,0.643,0.875,...,42.0,32.0,1610612744,113.0,0.494,0.760,0.364,32.0,36.0,1
7,2022-12-21,22200471,Final,1610612752,1610612761,2022,1610612752,106.0,0.553,0.611,...,25.0,38.0,1610612761,113.0,0.447,0.909,0.265,17.0,38.0,0
8,2022-12-21,22200472,Final,1610612745,1610612753,2022,1610612745,110.0,0.466,0.647,...,22.0,49.0,1610612753,116.0,0.451,0.697,0.297,19.0,45.0,0
9,2022-12-21,22200473,Final,1610612750,1610612742,2022,1610612750,99.0,0.494,0.700,...,23.0,39.0,1610612742,104.0,0.453,0.852,0.333,17.0,39.0,0


## 날짜 분포 확인하기

In [17]:
games_dates = set(games['GAME_DATE_EST'].values)
ranking_dates = set(ranking['STANDINGSDATE'].values)

len(games_dates), len(ranking_dates)

(4304, 7023)

In [16]:
preseason_cond = ranking['SEASON_ID'].str.startswith('1')
regular_cond = ranking['SEASON_ID'].str.startswith('2')

preseason_ranking = ranking[preseason_cond]
regular_ranking  = ranking[regular_cond]

In [18]:
preseason_dates = set(preseason_ranking['STANDINGSDATE'].values)
regular_dates = set(regular_ranking['STANDINGSDATE'].values)

len(preseason_dates), len(regular_dates)

(429, 6594)

In [23]:
games_dates_lst = sorted(list(games_dates))
preseason_dates_lst = sorted(list(preseason_dates))
regular_dates_lst = sorted(list(regular_dates))

In [45]:
import numpy as np
from typing import List

def get_continuous_dates(dates: List[np.datetime64]) -> List[list]:
    diff_arr = np.diff(dates)
    continuous_lst = []
    start = dates[0]
    for i, diff in enumerate(diff_arr):
        if diff > np.timedelta64(1, 'D'):
            end = dates[i]
            continuous_lst.append([start, end])
            start = dates[i+1]
    continuous_lst.append([start, dates[i]])
    return continuous_lst

In [46]:
games_continuous_dates = get_continuous_dates(games_dates_lst)
preseason_continuous_dates = get_continuous_dates(preseason_dates_lst)
regular_continuous_dates = get_continuous_dates(regular_dates_lst)

In [47]:
games_dates_data = [{'Category': 'Games', 'Start': start, 'End': end} for start, end in games_continuous_dates]
preseason_dates_data = [{'Category': 'Preseason', 'Start': start, 'End': end} for start, end in preseason_continuous_dates]
regular_dates_data = [{'Category': 'Regular', 'Start': start, 'End': end} for start, end in regular_continuous_dates]

dates_data = games_dates_data + preseason_dates_data + regular_dates_data

In [ ]:
import plotly.express as px

dates_data = pd.DataFrame(dates_data)

# 타임라인 차트
fig = px.timeline(
    dates_data,
    x_start="Start",
    x_end="End",
    y="Category",
    color="Category"
)

fig.update_layout(
    title="카테고리별 기간 타임라인",
    xaxis_title="날짜",
    yaxis_title="카테고리",
    width=900,
    height=400
)

fig.show()

### 문제 해결 - Start 와 End가 같으면 그래프에 없음


In [ ]:
dates_data.loc[dates_data['Start'] == dates_data['End'], 'End'] += pd.Timedelta(hours=1)

# 타임라인 차트
fig = px.timeline(
    dates_data,
    x_start="Start",
    x_end="End",
    y="Category",
    color="Category"
)

fig.update_layout(
    title="카테고리별 기간 타임라인",
    xaxis_title="날짜",
    yaxis_title="카테고리",
    width=900,
    height=400
)

fig.show()

In [55]:
season_dates = ranking.groupby('SEASON_ID').agg(Start=('STANDINGSDATE', 'min'), End=('STANDINGSDATE', 'max')).reset_index()

In [ ]:
# 타임라인 차트
fig = px.timeline(
    season_dates,
    x_start="Start",
    x_end="End",
    y="SEASON_ID",
    color="SEASON_ID"
)

fig.update_layout(
    title="시즌별 기간 타임라인",
    xaxis_title="날짜",
    yaxis_title="시즌",
    width=900,
    height=400
)

fig.show()

In [65]:
regular_ranking[regular_ranking['STANDINGSDATE'] == '2019-06']

,TEAM_ID,LEAGUE_ID,SEASON_ID,STANDINGSDATE,CONFERENCE,TEAM,G,W,L,W_PCT,HOME_RECORD,ROAD_RECORD,RETURNTOPLAY
95712,1610612744,0,22018,2019-06-01,West,Golden State,82,57,25,0.695,30-11,27-14,NaN
95713,1610612743,0,22018,2019-06-01,West,Denver,82,54,28,0.659,34-7,20-21,NaN
95714,1610612745,0,22018,2019-06-01,West,Houston,82,53,29,0.646,31-10,22-19,NaN
95715,1610612757,0,22018,2019-06-01,West,Portland,82,53,29,0.646,32-9,21-20,NaN
95716,1610612762,0,22018,2019-06-01,West,Utah,82,50,32,0.610,29-12,21-20,NaN
95717,1610612760,0,22018,2019-06-01,West,Oklahoma City,82,49,33,0.598,27-14,22-19,NaN
95718,1610612746,0,22018,2019-06-01,West,LA Clippers,82,48,34,0.585,26-15,22-19,NaN
95719,1610612759,0,22018,2019-06-01,West,San Antonio,82,48,34,0.585,32-9,16-25,NaN
95720,1610612758,0,22018,2019-06-01,West,Sacramento,82,39,43,0.476,24-17,15-26,NaN
95721,1610612747,0,22018,2019-06-01,West,L.A. Lakers,82,37,45,0.451,22-19,15-26,NaN


In [69]:
games[games['GAME_DATE_EST'] == '2019-05-23']

,GAME_DATE_EST,GAME_ID,GAME_STATUS_TEXT,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,...,AST_home,REB_home,TEAM_ID_away,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
19659,2019-05-23,41800305,Final,1610612749,1610612761,2018,1610612749,99.0,0.452,0.722,...,26.0,53.0,1610612761,105.0,0.369,0.806,0.419,19.0,45.0,0
